# fastcore.meta.FixSigMeta

## Imports

In [ ]:
from debuggable.utils import dbprint, defaults, whichversion, checksrc,\
defaults, colorize, insert2debug, dbsrclines, matchsrcorder, displaysrc, displaysavedbsrc

In [ ]:
# !pip install -U fastcore
whichversion('fastcore')

fastcore: 1.5.22 
Python supercharged for fastai development    
Jeremy Howard and Sylvain Gugger 
https://github.com/fastai/fastcore/     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore


In [ ]:
from fastcore.imports import *
from fastcore.meta import *
from fastcore.test import *
import inspect
from pprint import pprint
from datetime import date
import ast
import pickle

In [ ]:
today = date.today()
print("Today's date:", today)

Today's date: 2022-08-29


## Which source to explore

In [ ]:
defaults.name = "FixSigMeta"

## Upate my source

In [ ]:
# setup the folder and file when getting started. 
# with open("db/FixSigMetadb", "rb") as fp:   
#   defaults.src2dbp.FixSigMetadb = pickle.load(fp)

# with open("db/FixSigMeta", "rb") as fp:   
#   defaults.src2dbp.FixSigMeta = pickle.load(fp)

In [ ]:
checksrc()

class FixSigMeta(type):
    "A metaclass that fixes the signature on classes that override `__new__`"
    def __new__(cls, name, bases, dict):
        res = super().__new__(cls, name, bases, dict)
        if res.__init__ is not object.__init__: res.__signature__ = _rm_self(inspect.signature(res.__init__))
        return res

your debuggable srcode is empty. You have not written any, or you have lost your defaults.src2dbp.FixSigMetadb or db/FixSigMetadb file.


In [ ]:
# display exact the source codes you want to debug and mark them if they are already debuggable.
defaults.startsrc = "res = super()."
defaults.endsrc = "return res"
displaysrc()

class FixSigMeta(type):
    "A metaclass that fixes the signature on classes that override `__new__`"
    def __new__(cls, name, bases, dict):
        res = super().__new__(cls, name, bases, dict)                                                                                              ( 0 )     
        if res.__init__ is not object.__init__: res.__signature__ = _rm_self(inspect.signature(res.__init__))                                      ( 1 )     
        return res                                                                                                                                 ( 2 )     



## A personal docs on FixSigMeta

I have finished the prototype workflow of debuggable docs of fastai using `delegates`. `FixSigMeta` is the first important class I apply the workflow to.

### 1st step: run the official docs

read and run examples from the official [docs](https://hyp.is/8brb_B42Ee2vbjuvJtjGGw/nbviewer.org/github/fastai/fastcore/blob/master/nbs/07_meta.ipynb) and rewrite the docs and examples in my own words.

### When or why to use `FixSigMeta`?

When we want a class to use signature of `__init__` to be its class signature. 

In [ ]:
class Foo:
    def __init__(self, a, b, c): pass
    
inspect.signature(Foo)

<Signature (a, b, c)>

Many things can go wrong to prevent a class to use the signature from `__init__`. 

FixSigMeta is a metaclass, which helps us to get our classes' signature right.

Then what types of the signature problems can FixSigMeta fix?

1. when your class Foo inherits from class Base, if Base defines its `__new__`, then Foo can't get signature from `__init__`. (True for python 3.7 see [demos](https://www.kaggle.com/code/danielliao/notebook3edc928f49?scriptVersionId=104385507&cellId=1), no more for 3.9+)

In [ ]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base):
    def __init__(self, d, e, f): pass
    
inspect.signature(Foo) # no more problem for python 3.9+, 

<Signature (d, e, f)>

In [ ]:
from IPython.display import IFrame

In [ ]:
IFrame(src="https://www.kaggle.com/embed/danielliao/notebook3edc928f49?cellIds=2&kernelSessionId=104407182", width = "1200", height="300", \
       style="margin: 0 auto; width: 100%; max-width: 950px;", frameborder="0", scrolling="auto", title="notebook3edc928f49")

Solution to 1: By also inheriting from the metaclass FixSigMeta can solve the signature problem for Foo (for python 3.7)

In [ ]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base, metaclass=FixSigMeta):
    def __init__(self, d, e, f): pass
    
test_sig(Foo, '(d, e, f)')
inspect.signature(Foo)

<Signature (d, e, f)>

In [ ]:
IFrame(src="https://www.kaggle.com/embed/danielliao/notebook3edc928f49?cellIds=4&kernelSessionId=104407182", width = "1200", height="300", \
       style="margin: 0 auto; width: 100%; max-width: 950px;", frameborder="0", scrolling="auto", title="notebook3edc928f49")

2. when your Foo has a metaclass BaseMeta, if BaseMeta need to define its `__call__`, then Foo can't get signature from `__init__`.

In [ ]:
class BaseMeta(type): 
    # using __new__ from type
    def __call__(cls, *args, **kwargs): pass
class Foo(metaclass=BaseMeta): 
    def __init__(self, d, e, f): pass

test_sig(Foo, '(*args, **kwargs)')

In [ ]:
class BaseMeta(type): 
    def __new__(cls, name, bases, dict):
        return super().__new__(cls, name, bases, dict) # using __new__ from type
    def __call__(cls, *args, **kwargs): pass
class Foo(metaclass=BaseMeta): 
    def __init__(self, d, e, f): pass

test_sig(Foo, '(*args, **kwargs)')

Solution to problem 2: you need to inherit from FixSigMeta instead of type when constructing the metaclass to preserve the signature in `__init__`. Be careful not to override `__new__` when doing this:

In [ ]:
class BaseMeta(FixSigMeta): 
    # using __new__ of  FixSigMeta instead of type
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')

In [ ]:
class BaseMeta(FixSigMeta): 
    def __new__(cls, name, bases, dict): # not really overriding __new__, but still using FixSigMeta.__new__
        return super().__new__(cls, name, bases, dict)
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')

Note: if Base also defines `__new__`, then FixSigMeta can't help. 

In [ ]:
class BaseMeta(FixSigMeta): 
    # __new__ comes from FixSigMeta
    def __new__(cls, *args, **kwargs): pass # actually overriding __new__, not using FixSigMeta.__new__ anymore
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_eq(type(Foo), type(None))

Note: if Base also defines `__init__`, then FixSigMeta can still help. 

In [ ]:
class BaseMeta(FixSigMeta): 
    # __new__ comes from FixSigMeta
    def __init__(cls, *args, **kwargs): pass # overriding __init__ of FixSigMeta, is fine
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')

## Debugging `inspect.signature(Foo)` to find out how Foo use signature from `__init__`

In [ ]:
class Foo:
    def __init__(self, a, b, c): pass
    
inspect.signature(Foo)

<Signature (a, b, c)>

### Finding out the real source code for `inspect.signature`

In [ ]:
def signature(obj, *, follow_wrapped=True):
    """Get a signature object for the passed callable."""
    return Signature.from_callable(obj, follow_wrapped=follow_wrapped)

In [ ]:
inspect.Signature.from_callable??

Signature: inspect.Signature.from_callable(obj, *, follow_wrapped=True)
Source:   
    @classmethod
    def from_callable(cls, obj, *, follow_wrapped=True):
        """Constructs Signature for the given callable object."""
        return _signature_from_callable(obj, sigcls=cls,
                                        follow_wrapper_chains=follow_wrapped)
File:      ~/mambaforge/lib/python3.9/inspect.py
Type:      method


In [ ]:
inspect._signature_from_callable(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True)

<Signature (a, b, c)>

### Debugging `inspect._signature_from_callable`

In [ ]:
def _signature_from_callable(obj, *,
                             follow_wrapper_chains=True,
                             skip_bound_arg=True,
                             sigcls):

    """Private helper function to get signature for arbitrary
    callable objects.
    """

    _get_signature_of = functools.partial(_signature_from_callable,
                                follow_wrapper_chains=follow_wrapper_chains,
                                skip_bound_arg=skip_bound_arg,
                                sigcls=sigcls)

    if not callable(obj):
        raise TypeError('{!r} is not a callable object'.format(obj))

    if isinstance(obj, types.MethodType):
        # In this case we skip the first parameter of the underlying
        # function (usually `self` or `cls`).
        sig = _get_signature_of(obj.__func__)

        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    # Was this function wrapped by a decorator?
    if follow_wrapper_chains:
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))
        if isinstance(obj, types.MethodType):
            # If the unwrapped object is a *method*, we might want to
            # skip its first parameter (self).
            # See test_signature_wrapped_bound_method for details.
            return _get_signature_of(obj)

    try:
        sig = obj.__signature__
    except AttributeError:
        pass
    else:
        if sig is not None:
            if not isinstance(sig, Signature):
                raise TypeError(
                    'unexpected object {!r} in __signature__ '
                    'attribute'.format(sig))
            return sig

    try:
        partialmethod = obj._partialmethod
    except AttributeError:
        pass
    else:
        if isinstance(partialmethod, functools.partialmethod):
            # Unbound partialmethod (see functools.partialmethod)
            # This means, that we need to calculate the signature
            # as if it's a regular partial object, but taking into
            # account that the first positional argument
            # (usually `self`, or `cls`) will not be passed
            # automatically (as for boundmethods)

            wrapped_sig = _get_signature_of(partialmethod.func)

            sig = _signature_get_partial(wrapped_sig, partialmethod, (None,))
            first_wrapped_param = tuple(wrapped_sig.parameters.values())[0]
            if first_wrapped_param.kind is Parameter.VAR_POSITIONAL:
                # First argument of the wrapped callable is `*args`, as in
                # `partialmethod(lambda *args)`.
                return sig
            else:
                sig_params = tuple(sig.parameters.values())
                assert (not sig_params or
                        first_wrapped_param is not sig_params[0])
                new_params = (first_wrapped_param,) + sig_params
                return sig.replace(parameters=new_params)

    if isfunction(obj) or _signature_is_functionlike(obj):
        # If it's a pure Python function, or an object that is duck type
        # of a Python function (Cython functions, for instance), then:
        return _signature_from_function(sigcls, obj,
                                        skip_bound_arg=skip_bound_arg)

    if _signature_is_builtin(obj):
        return _signature_from_builtin(sigcls, obj,
                                       skip_bound_arg=skip_bound_arg)

    if isinstance(obj, functools.partial):
        wrapped_sig = _get_signature_of(obj.func)
        return _signature_get_partial(wrapped_sig, obj)

    
    dbprint("    if isinstance(obj, type):", "our example Foo is a simple class", "isinstance(obj, type)", obj=obj)
    sig = None
    if isinstance(obj, type):
        # obj is a class or a metaclass

        # First, let's see if it has an overloaded __call__ defined
        # in its metaclass
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            sig = _get_signature_of(call)
        else:
            factory_method = None
            new = _signature_get_user_defined_method(obj, '__new__')
            init = _signature_get_user_defined_method(obj, '__init__')
            # Now we check if the 'obj' class has an own '__new__' method
            if '__new__' in obj.__dict__:
                factory_method = new
            # or an own '__init__' method
            elif '__init__' in obj.__dict__:
                factory_method = init
            # If not, we take inherited '__new__' or '__init__', if present
            elif new is not None:
                factory_method = new
            elif init is not None:
                factory_method = init

            if factory_method is not None:
                sig = _get_signature_of(factory_method)

        if sig is None:
            # At this point we know, that `obj` is a class, with no user-
            # defined '__init__', '__new__', or class-level '__call__'

            for base in obj.__mro__[:-1]:
                # Since '__text_signature__' is implemented as a
                # descriptor that extracts text signature from the
                # class docstring, if 'obj' is derived from a builtin
                # class, its own '__text_signature__' may be 'None'.
                # Therefore, we go through the MRO (except the last
                # class in there, which is 'object') to find the first
                # class with non-empty text signature.
                try:
                    text_sig = base.__text_signature__
                except AttributeError:
                    pass
                else:
                    if text_sig:
                        # If 'base' class has a __text_signature__ attribute:
                        # return a signature based on it
                        return _signature_fromstr(sigcls, base, text_sig)

            # No '__text_signature__' was found for the 'obj' class.
            # Last option is to check if its '__init__' is
            # object.__init__ or type.__init__.
            if type not in obj.__mro__:
                # We have a class (not metaclass), but no user-defined
                # __init__ or __new__ for it
                if (obj.__init__ is object.__init__ and
                    obj.__new__ is object.__new__):
                    # Return a signature of 'object' builtin.
                    return sigcls.from_callable(object)
                else:
                    raise ValueError(
                        'no signature found for builtin type {!r}'.format(obj))

    elif not isinstance(obj, _NonUserDefinedCallables):
        # An object with __call__
        # We also check that the 'obj' is not an instance of
        # _WrapperDescriptor or _MethodWrapper to avoid
        # infinite recursion (and even potential segfault)
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            try:
                sig = _get_signature_of(call)
            except ValueError as ex:
                msg = 'no signature found for {!r}'.format(obj)
                raise ValueError(msg) from ex

    if sig is not None:
        # For classes and objects we skip the first parameter of their
        # __call__, __new__, or __init__ methods
        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    if isinstance(obj, types.BuiltinFunctionType):
        # Raise a nicer error message for builtins
        msg = 'no signature found for builtin function {!r}'.format(obj)
        raise ValueError(msg)

    raise ValueError('callable {!r} is not supported by signature'.format(obj))

In [ ]:
class Foo:
    def __init__(self, a, b, c): pass

from inspect import *
from inspect import _signature_is_functionlike, _signature_is_builtin
# rewrite inspect.signature(Foo) in the following way for debugging easily here
_signature_from_callable(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True)



######################################################## source code with lines under investigation #########################################################


class FixSigMeta(type):                                                                                                                                      
    "A metaclass that fixes the signature on classes that override `__new__`"                                                                                
    def __new__(cls, name, bases, dict):                                                                                                                     
        res = super().__new__(cls, name, bases, dict)                                                                                                        
        if res.__init__ is not object.__init__: res.__signature__ = _rm_self(inspect.signature(res.__init__))                                                
        return res                              

TypeError: unsupported format string passed to NoneType.__format__

### debugging the error to understand the source

## Add srcline and dbprints

In [ ]:
# with open("db/delegates", "rb") as fp:   # Unpickling
#   defaults.src2dbp.delegates = pickle.load(fp)

In [ ]:
# srcdbps = defaults.src2dbp.delegates # user input
srcdbps = []

In [ ]:
displaysrc()

class FixSigMeta(type):
    "A metaclass that fixes the signature on classes that override `__new__`"
    def __new__(cls, name, bases, dict):
        res = super().__new__(cls, name, bases, dict)                                                                                              ( 0 )     
        if res.__init__ is not object.__init__: res.__signature__ = _rm_self(inspect.signature(res.__init__))                                      ( 1 )     
        return res                                                                                                                                 ( 2 )     



In [ ]:
srcline = """
        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
"""

In [ ]:
len(defaults.src.split(srcline)) # must be 2 to be right

2

In [ ]:
dbcode = """
        dbprint("        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items() if v.default != inspect.Parameter.empty and k not in sigd and k not in but}", \
"what it does: f take the params from to and put them into a dict, k is name, v is the param in Parameter class, and make their kind KEYWORD_ONLY.", \
"if callable(to_f):\\n\
    if bool(inspect.signature(to_f).parameters.items()):\\n\
        for k,v in inspect.signature(to_f).parameters.items():\\n\
            print(f'k:v => {k}:{v}, type(v): {type(v)}')\\n\
    else:\\n\
        print(f'inspect.signature(to_f).parameters.items() is empty: {not bool(inspect.signature(to_f).parameters.items())}')\\n\
else:\\n\
    print(f'to_f is callable: {callable(to_f)}, so inspect.signature(to_f) will cause error')",\
to_f=to_f, k=k, but=but, sigd=sigd)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = "        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}"
dbcode = """
        dbprint("        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}", \
"what it does: check to_f's annotations (dict), and only select params with their annotations wanted by f or from_f.\\n\
what is __annotations__: a:int, b:int=1 are annotations\\n\
for k,v in to_f.__annotations__.items(): print(f'k:v => {k}:{v}')\\n\
    if the iterator above is empty, then print won't get executed.\\n\
    run this:\\n\
    for k, v in {}.items(): print(f'k:v => {k}:{v}') ", \
"hasattr(to_f, '__annotations__')", \
"if hasattr(to_f, '__annotations__'):\\n\
    if bool(to_f.__annotations__.items()) == False:\\n\
        print(f'to_f.__annotations__.items(): {to_f.__annotations__.items()}')\\n\
    else:\\n\
        for k, v in to_f.__annotations__.items():\\n\
            print(f'k:v => {k}:{v}')\\n\
else:\\n\
    print(f'We should expect error from the code of try-except.')\\n\
    try:\\n\
        to_f.__annotations__\\n\
    except AttributeError as e:\\n\
        print(e)", \
but=but, k=k, sigd=sigd, to_f=to_f)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = """
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
"""

# watch out: to convert srcline into actual strings for the first arg in dbprint, use '\\n\' instead of just '\n'
dbcode = """
\n        dbprint("        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\\n\        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f",\
"UseCase1: when to is superclass to f\\nUseCase2: when to is a class but not superclass to f. UseCase3: other combinations",\
"if to is None:\\n\
    to_f,from_f = f.__base__.__init__,f.__init__\\n\
else:\\n\
    to_f,from_f = to.__init__ if isinstance(to,type) else to,f",\
"to_f", "from_f", "hasattr(to_f, '__annotations__')", "hasattr(from_f, '__annotations__')", \
f=f, to=to)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
srcline = "        if hasattr(from_f,\'__delwrap__\'): return f"
dbcode = """
        dbprint("if hasattr(from_f,'__delwrap__'): return f", \
"If f or from_f has __delwrap__, it means it's happy with all params and give up on **kwargs. So no more params needed from other `to`s", \
"from_f", "f", "hasattr(from_f, '__delwrap__')", "if hasattr(from_f,'__delwrap__'): return f", from_f=from_f, f=f)
"""

In [ ]:
srcdbps.append([(srcline, dbcode)])

In [ ]:
# Watch out: using \\n instead of \n inside a block
srcline = """
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
"""
dbcode = """
\n        dbprint("        from_f = getattr(from_f,'__func__',from_f)\\n\        to_f = getattr(to_f,'__func__',to_f)",\
"This line is for classmethod, as it is not callable, so inspect.signature(...) won't work, but it has __func__ to save",\
"f", "from_f", "type(from_f)",\
"try:\\n\
    inspect.signature(from_f)\\n\
except:\\n\
    print('error occurs')\\n\
    print(f'is from_f callable: {callable(from_f)}')\\n\
else:\\n\
    print(inspect.signature(from_f))", \
"hasattr(from_f, '__func__')", "from_f = getattr(from_f,'__func__',from_f)", "from_f",\
"to", "to_f", "type(to_f)",\
"try:\\n\
    inspect.signature(to_f)\\n\
except:\\n\
    print('error occurs')\\n\
    print(f'is to_f callable: {callable(to_f)}')\\n\
else:\\n\
    print(inspect.signature(to_f))", \
"hasattr(to_f, '__func__')", "to_f = getattr(to_f,'__func__',to_f)", "to_f",\
from_f=from_f, to_f=to_f, f=f, to=to, srcline=srcline)
"""
# Importance: must include all necessary env into the dbprint function above. Note, we don't need inspect=inspect because utils has exported `import inspect`

In [ ]:
srcdbps.append([(srcline, dbcode)])

## Match scrdbps with right order

In [ ]:
for s in srcdbps:
    print(s[0][0])


        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}

        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}

        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f

        if hasattr(from_f,'__delwrap__'): return f

        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)



In [ ]:
srcdbps = matchsrcorder(srcdbps)

In [ ]:
for s in srcdbps:
    print(s[0][0])


        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f


        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)

        if hasattr(from_f,'__delwrap__'): return f

        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}

        anno = {k:v for k,v in to_f.__annotations__.items() if k not in sigd and k not in but}


## Save the debuggable source code

In [ ]:
# save all the srclines with their dbcodes into defaults.
defaults.src2dbp.delegates = srcdbps

In [ ]:
# save the latest debuggable source code into defaults.src2dbp.delegatesdb
# dbsrclines(dbsrc=True) 
displaysavedbsrc()

def delegates(to:FunctionType=None, # Delegatee
              keep=False, # Keep `kwargs` in decorated function?
              but:list=None): # Exclude these parameters from signature
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
    def _f(f):

        dbprint("        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\n\        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f","UseCase1: when to is superclass to f\nUseCase2: when to is a class but not superclass to f. UseCase3: other combinations","if to is None:\n    to_f,from_f = f.__base__.__init__,f.__init__\nelse:\n    to_f,from_f = to.__init__ if isinstance(to,type) else to,f","to_f", "from_f", "hasattr(to_f, '__annotations__')", "hasattr(from_f, '__annotations__')", f=f, to=to)

        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f

        dbpr

In [ ]:
with open("db/delegates", "wb") as fp:   
  pickle.dump(defaults.src2dbp.delegates, fp)

with open("db/delegatesdb", "wb") as fp:   
  pickle.dump(defaults.src2dbp.delegatesdb, fp)

## Debugging one or more srclines

In [ ]:
displaysrc() # (0) refers to official srcline, =0= refer to the list index of tuple (srcline, dbprint) inside defaults.src2dbps.delegates

def delegates(to:FunctionType=None, # Delegatee
              keep=False, # Keep `kwargs` in decorated function?
              but:list=None): # Exclude these parameters from signature
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__================================================================================( 0 )==0==
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f==================================================================( 1 )==0==
        from_f = getattr(from_f,'__func__',from_f)=================================================================================================( 2 )==1==
        to_f = getattr(to_f,'__func__',to_f)=======================================================================================================( 3 )==1==
        if hasattr(from_f,'__delwrap__'): return f==================

To debug examples with errors

In [ ]:
delegates = dbsrclines([0,3], retn=True) # print out the debuggable source code under investgation

def delegates(to:FunctionType=None, # Delegatee
              keep=False, # Keep `kwargs` in decorated function?
              but:list=None): # Exclude these parameters from signature
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
    def _f(f):

        dbprint("        if to is None: to_f,from_f = f.__base__.__init__,f.__init__\n\        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f","UseCase1: when to is superclass to f\nUseCase2: when to is a class but not superclass to f. UseCase3: other combinations","if to is None:\n    to_f,from_f = f.__base__.__init__,f.__init__\nelse:\n    to_f,from_f = to.__init__ if isinstance(to,type) else to,f","to_f", "from_f", "hasattr(to_f, '__annotations__')", "hasattr(from_f, '__annotations__')", f=f, to=to)

        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_

In [ ]:
# delegates = dbsrclines([3]) # print out the debuggable source code under investgation

In [ ]:
defaults.eg = """
class Base(): 
    def __init__(self, a, b=1): pass 
class Other():
    def __init__(self, e=1): pass

@delegates()
class Subcls(Base): # to as a superclass (Base),  to must have __init__, and __new__ won't do
    def __init__(self, c, d=1, **kwargs): pass # f as class, should have __init__
test_sig(Subcls, '(c, d=1, *, b=1)')
"""

class Base(): 
    def __init__(self, a, b=1): pass 
class Other():
    def __init__(self, e=1): pass

@delegates()
class Subcls(Base): # to as a superclass (Base),  to must have __init__, and __new__ won't do
    def __init__(self, c, d=1, **kwargs): pass # f as class, should have __init__
# test_sig(Subcls, '(c, d=1, *, b=1)')



######################################################## source code with lines under investigation #########################################################


def delegates(to:FunctionType=None, # Delegatee                                                                                                              
              keep=False, # Keep `kwargs` in decorated function?                                                                                             
              but:list=None): # Exclude these parameters from signature                                                                                      
    "Decorator: replace `**kwargs` in signature with params from `to`"                                                                                       
    if but is None: but = []                                                                                                                                 
    def _f(f):                                  

In [ ]:
defaults.eg = """
class Base(): 
    def __new__(self, a, b=1): pass 
class Other():
    def __init__(self, e=1): pass

@delegates()
class Subcls(Base): # to as a superclass (Base),  to must have __init__, and __new__ won't do
    def __init__(self, c, d=1, **kwargs): pass # f as class, should have __init__
test_sig(Subcls, '(c, d=1, *, b=1)')
"""

class Base(): 
    def __new__(self, a, b=1): pass 
class Other():
    def __init__(self, e=1): pass

@delegates()
class Subcls(Base): # to as a superclass (Base),  to must have __init__, and __new__ won't do
    def __init__(self, c, d=1, **kwargs): pass # f as class, should have __init__
# test_sig(Subcls, '(c, d=1, *, b=1)')



######################################################## source code with lines under investigation #########################################################


def delegates(to:FunctionType=None, # Delegatee                                                                                                              
              keep=False, # Keep `kwargs` in decorated function?                                                                                             
              but:list=None): # Exclude these parameters from signature                                                                                      
    "Decorator: replace `**kwargs` in signature with params from `to`"                                                                                       
    if but is None: but = []                                                                                                                                 
    def _f(f):                                  

#|hide
## Sending to Obs

In [ ]:
#|hide
!jupytext --to md /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.ipynb
!mv /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/fastcore/meta/

[jupytext] Reading /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.ipynb in format ipynb
[jupytext] Writing /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.md


In [ ]:
#|hide
!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert

[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/index.ipynb to markdown
[NbConvertApp] Writing 2074 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/index.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/utils.ipynb to markdown
[NbConvertApp] Writing 190008 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/utils.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/classes_metaclasses.ipynb to markdown
[NbConvertApp] Writing 23234 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/classes_metaclasses.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/meta/02_use_kwargs_dict.ipynb to markdown
[NbConvertApp] Writing 3754 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/02_use_kwargs_dict.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/meta/00_delegates.ipynb to markdown
[NbConvertAp